# Getting started with the Java API
The API can be added to your project by either downlowding the jar file or adding the following dependency to your pom.xml: 

In [1]:
%%loadFromPOM
<dependency>
  <groupId>de.tuebingen.uni.sfs.germanet</groupId>
  <artifactId>germanet-api</artifactId>
  <version>13.2.1</version>
</dependency>

The API does some logging, for example when the GermaNet data is loaded, each XML-file will be logged once loaded. If your project does not include any logging library (e.g. logback-classic) you will get a warning message. If you want to see the logs you can add this dependency:

```
<dependency>
    <groupId>ch.qos.logback</groupId>
    <artifactId>logback-classic</artifactId>
    <version>1.2.3</version>
</dependency`
```

Once you added the GermaNet dependency to your project, you can import the API:

In [2]:
import de.tuebingen.uni.sfs.germanet.api.*;

Whenever you want to use the API, the first thing you would do is to create a GermaNet object as this loads the data and provides access to it. The data (all XML files) have to be stored in one directory which has to be specified as the first argument when you construct the GermaNet object. If you want to run this code, put your XML files in a "germanet_data" directory in your home directory or change the path to the location on your computer.
The API also provides methods to compute semantic similarity / relatedness between words (Synsets). To be able to use all of them you have to provide frequency lists for each word category. These lists can be downloaded from:



In [3]:
String userHome = System.getProperty("user.home");
String data_path = userHome+"/germanet/GN_V150/GN_V150_XML";
String freq_list_path = userHome+"/germanet/GN_V150/FreqLists/";
String nounFreqListPath = freq_list_path + "/noun_freqs_decow14_16.txt";
String verbFreqListPath = freq_list_path + "/verb_freqs_decow14_16.txt";
String adjFreqListPath = freq_list_path + "/adj_freqs_decow14_16.txt";
System.out.println(data_path);
GermaNet germanet = new GermaNet(data_path, nounFreqListPath, verbFreqListPath, adjFreqListPath);
System.out.println("Germanet loaded.")

/Users/nwitte/germanet/GN_V150/GN_V150_XML


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Germanet loaded.


The data has been loaded and we can now use the API to extract specific information from the data.

# How to inspect information for a single word input
## Inspect synsets
Let's consider the input word Fußball 'football'. The following shows how to extract all synsets given an input word. Many words are ambiguous; Fußball belongs to two synsets.

In [4]:
List <Synset> fussball_synsets = germanet.getSynsets("Fußball");
// the length of the retrieved list is equal to the number of possible senses for a word, in this case 2
fussball_synsets.size();

2

The string representations include the lexical units, which can be helpful when you want to select a specific meaning for a given word.

In [5]:
for (Synset synset : fussball_synsets){
    System.out.println(synset.toString());
}

id: 21624, orth forms: [Fußball, Fußballspiel, Fußballsport], paraphrases: Sport, Freizeit, kein Plural: eine beliebte Mannschaftssportart, welche mit 22 Spielern und einem Ball gespielt wird
id: 7944, orth forms: [Fußball], paraphrases: Sport, Freizeit: ein Ball zur Ausübung des Sportes


In this case, let's say we are interested in the sense of *Fußball* which is synonymous with *Fußballspiel* &mdash; that is, the game rather than the ball.

In [6]:
Synset fussball_synset = germanet.getSynsetByID(21624);

Every synset has a number of properties that can be extracted. Each synset has a unique id.

In [7]:
fussball_synset.getId();


21624

A synset can have one of three possible word categories (verb, noun, adjective). 

In [8]:
fussball_synset.getWordCategory();

nomen

For each of the word categories the semantic space is divided into a number of semantic fields. (e.g *Besitz*,
*Kommunikation*, *Geschehen*...), called `WordClass`. 

In [9]:
fussball_synset.getWordClass();

Geschehen

Synsets are related to other synsets via conceptual relations `ConRel`. The most important relation is the hypernymy
/ hyponymy relation. Direct hypernyms of a synset (one level above) and hyponyms (one level below) can be accessed through the relations `has_hypernym` and `has_hyponym`.

In [10]:
fussball_synset.getRelatedSynsets(ConRel.has_hypernym);

[id: 21606, orth forms: [Ballspiel, Ballsportart, Ballsport], paraphrases: Ein Spiel, das mit einem Ball gespielt wird; Eine Sportart, die mit einem Ball betrieben wird]

In [11]:
fussball_synset.getRelatedSynsets(ConRel.has_hyponym);

[id: 71210, orth forms: [Jugendfußball], id: 69685, orth forms: [Amateurfußball], id: 84590, orth forms: [Männerfußball], id: 21625, orth forms: [Profifußball], id: 21626, orth forms: [Damenfußball, Frauenfußball], id: 137940, orth forms: [Berufsfußball], id: 139802, orth forms: [Torwandschießen], id: 133871, orth forms: [Mädchenfußball], id: 146603, orth forms: [Straßenfußball], id: 145820, orth forms: [Kombinationsfußball], id: 137475, orth forms: [Klubfußball, Clubfußball], id: 79925, orth forms: [Hallenfußball], id: 104374, orth forms: [Spitzenfußball], id: 62081, orth forms: [Vereinsfußball], id: 133731, orth forms: [Weltfußball], id: 124051, orth forms: [Freizeitfußball]]

We can see that *Fußball* has exactly one hypernym and several hyponyms. It is also possible to list all <!-- transitive? --> hypernyms
from *Fußball* to the top node (root node).


In [12]:
Set all_hypernyms = new HashSet();
List<List<Synset>> transitive_closure = fussball_synset.getTransRelatedSynsets(ConRel.has_hypernym);
for (List<Synset> p : transitive_closure){
    for (Synset hypernym: p){
        if (!hypernym.equals(fussball_synset)) {
        all_hypernyms.add(hypernym.getAllOrthForms());
        }
    }
}

all_hypernyms;


[[Veranstaltung], [qualitative Beziehung], [Sport, Sportdisziplin, Sportart, Disziplin], [Ereignis], [Ballspiel, Ballsportart, Ballsport], [Verhältnis, Relation, Beziehung], [Situation], [Wettkampf, Kampf], [GNROOT], [Tat, Tätigkeit, Handlung, Aktivität], [Sportwettkampf], [Zustand], [Spiel, Sportspiel, Partie, Match], [Geschehen, Geschehnis], [Sportereignis, Sportveranstaltung], [Konflikt, Auseinandersetzung]]

The level where the *Fußball* synset is attached to the Graph is called its depth.

In [13]:
fussball_synset.getDepth();

8

## Use the semantic utils to measure semantic similarity / relatedness

You can also use the API to compare a synset with another synset. These methods work only for two synsets that have the same word category, for example for two nouns. There are two different types of similarity measures:
- path-based measures
- information-content-based measures

Path-based measures compute the semantic relatedness between two concepts based on the shortest path between two synsets in the hypernym relation. The shortest path length is the minimal number of nodes forming a path between the two synsets in the relation. Different measures weigh or normalize the path-length in different ways.


### Example for path-based measures

The following example shows how to use path-based semantic relatedness measures to check whether *Trompete* (trumpet) is more closely related to *Posaune* (trombone) than to *Flöte* (flute) and how to disambiguate *Flügel* (wing, blade, grand) in the context of *Klavier* (piano). 

To use the path-based semantic relatedness measures you have to initialize a `SemanticUtils` object.

As mentioned above, these measures only work for synsets that belong to the same word category.

In [14]:
SemanticUtils semanticUtils = germanet.getSemanticUtils();


We can now use the `SemanticUtils` object to find out whether *Trompete* (trumpet) is more closely related to *Posaune* (trombone) or to *Flöte* (flute):

In [15]:
Synset trompete = germanet.getSynsets("Trompete").get(0);
trompete;

id: 11590, orth forms: [Trompete], paraphrases: Musik: Hohes Blechblasinstrument mit einem Kesselmundstück

In [16]:
Synset flöte = germanet.getSynsets("Flöte").get(0);
flöte;

id: 11572, orth forms: [Flöte], paraphrases: ein Blasinstrument, ein Musikinstrument

In [17]:
Synset posaune = germanet.getSynsets("Posaune").get(0);
posaune;

id: 11589, orth forms: [Posaune], paraphrases: Musik: ein Blechblasinstrument mit einem Zug zum verändern des Tones

In [18]:
Double trompete_posaune = semanticUtils.getSimilaritySimplePath(trompete, posaune, 10);
Double trompete_flöte = semanticUtils.getSimilaritySimplePath(trompete, flöte, 10);
trompete_posaune > trompete_flöte;

true

Path based relatedness measures can also be used to disambiguate word senses. This example shows how to find the sense of *Flügel* (wing, blade, grand) which is most similar to *Klavier* (piano) according to three different path-based measures:. 

In [19]:
%%loadFromPOM
<dependency>
    <groupId>de.vandermeer</groupId>
    <artifactId>asciitable</artifactId>
    <version>0.3.2</version>
</dependency>

In [20]:
Synset klavier = germanet.getSynsets("Klavier").get(0);
List<Synset> flügel_synsets = germanet.getSynsets("Flügel");
klavier;

id: 11619, orth forms: [Piano, Klavier, Pianoforte], paraphrases: Tasteninstrument, dessen Klang durch Saiten erzeugt wird; Klavier, Abkürzung für Pianoforte. (Am Klavier kann man im Gegensatz zum Cembalo die Dynamik verändern.)

In [21]:
import de.vandermeer.asciitable.*;
AsciiTable at = new AsciiTable();
at.addRule();
at.addRow(" Synset ID ", " Orth Forms ", " Simple Path ", " Leacock and Chodorow ", " Wu and Palmer ");
at.addRule();
Double highest_sim_simple = 0.0;
Double highest_sim_leacock = 0.0;
Double highest_sim_wu = 0.0;
Synset most_similar_synset = null;
for (Synset synset : flügel_synsets){
    if (synset.getWordCategory().equals(WordCategory.nomen)){
        Double sim_simple = semanticUtils.getSimilarity(SemRelMeasure.SimplePath, synset, klavier, 1);
        Double sim_leacock = semanticUtils.getSimilarity(SemRelMeasure.LeacockAndChodorow, synset, klavier, 1);
        Double sim_wu = semanticUtils.getSimilarity(SemRelMeasure.WuAndPalmer, synset, klavier, 1);
        at.addRow(String.format("%d", synset.getId()), synset.getAllOrthForms().toString(), String.format("%.2f", sim_simple), String.format("%.2f", sim_leacock), String.format("%.2f", sim_wu));
        //System.out.println(String.format("\n These are the similarities between the synset for Klavier and %s : \n Simple Path : %.2f\n Leackock and Chodorow: %.2f\n Wu and Palmer : %.2f", synset.toString(), sim_simple, sim_leacock, sim_wu));
        if (sim_simple > highest_sim_simple && sim_leacock > highest_sim_leacock && sim_wu > highest_sim_wu ){
            highest_sim_simple = sim_simple;
            highest_sim_leacock = sim_leacock;
            highest_sim_wu = sim_wu;
            most_similar_synset = synset;
            }
        }
}
most_similar_synset;

id: 11625, orth forms: [Flügel], paraphrases: Musik, Musikinstrumentenbau: ein großes Klavier, besonders für Konzerte, dessen Deckel meist geöffnet ist, wenn man darauf spielt

We can verify that this is the most similar synset by looking at all the similarity results in a table:

In [22]:
at.addRule();
at.setPaddingLeftRight(1);
String rend = at.render();
System.out.println(rend);

┌───────────────┬───────────────┬───────────────┬───────────────┬──────────────┐
│ Synset ID     │ Orth Forms    │ Simple Path   │ Leacock   and │ Wu       and │
│               │               │               │ Chodorow      │ Palmer       │
├───────────────┼───────────────┼───────────────┼───────────────┼──────────────┤
│ 73683         │ [Flügel]      │ 0,69          │ 0,31          │ 0,35         │
│ 12102         │ [Flügel,      │ 0,80          │ 0,42          │ 0,53         │
│               │ Rotorblatt]   │               │               │              │
│ 9697          │ [Flügel,      │ 0,74          │ 0,36          │ 0,40         │
│               │ Seitenflügel] │               │               │              │
│ 26446         │ [Flügel,      │ 0,69          │ 0,31          │ 0,27         │
│               │ Schwinge]     │               │               │              │
│ 23151         │ [Flügel,      │ 0,66          │ 0,28          │ 0,33         │
│               │ Parteiflüg

### Example for IC-based measures

One problem with path-based measures is that paths of the same length in the hypernym relation can correspond to very different intuitive semantic "distances".
Measures based on *information content* (IC) seek to solve this problem by augmenting information about the structural distances in the hypernym relation with information about word frequencies. 
The word frequencies are used to compute the information content, which grades concepts from more specific to more general. If a very specific synset is compared to a very general one, the relatedness will be low. The relatedness of two synsets is measured based on the information content of their least common subsumer (the lowest synset in the hierarchy that is hypernym to both synsets).

To use these measures, you have to create download frequency lists for words in Germanet. These lists contain the raw frequencies of the nouns, adjectives and verbs that are in Germanet, based on a very large corpus. You can use either the provided frequency lists or your own lists.

The following code snippet shows the advantage of the IC-based measures. While path-based measures would classify the words *Pflanze* 'plant' and *Tier* 'animal' as being almost as similar as the words *Roteiche* 'red oak' and *Steineiche* 'holm oak', the IC-based measures distinguish whether two synsets are very general or more specific and consequently assign a higher similarity score to the second pair of words.

In [23]:
//first word pair:
Synset pflanze = germanet.getSynsetByID(44960);
Synset tier = germanet.getSynsetByID(48805);

//second word pair:
Synset roteiche = germanet.getSynsetByID(46054);
Synset steineiche = germanet.getSynsetByID(46056);

Notice that a path-based measure between these word pairs yields almost the same results:



In [24]:
semanticUtils.getSimilarity(SemRelMeasure.LeacockAndChodorow, pflanze, tier, 1);

0.6131471927654584

In [25]:
semanticUtils.getSimilarity(SemRelMeasure.LeacockAndChodorow, roteiche, steineiche, 1);

0.6934264036172707

But an IC-based measure clearly distinguishes the two pairs:

In [26]:
semanticUtils.getSimilarity(SemRelMeasure.Resnik, pflanze, tier, 1);

0.10984810485167636

In [27]:
semanticUtils.getSimilarity(SemRelMeasure.Resnik, roteiche, steineiche, 1);

0.5159160829148369

For a more convenient search through the ontology and the semantic relatedness computation, you can use the GermaNet web application "Rover":
https://weblicht.sfs.uni-tuebingen.de/rover/

## Inspect Lexical Units
Every synset contains one ore several Lexical Units. The list of Lexical Units (lexunit) can be accessed for any synset. Let's inspect the lexical units for *Fußball* 'football':
We have the lexunit *Fußballspiel* 'football match', the lexunit *Fußball* 'football' and the lexunit *Fußballsport* 'soccer'.

In [28]:
fussball_synset.getLexUnits();


[id: 29778, orth form: Fußballsport, synset id: 21624, sense: 1, source: core, named entity: false, artificial: false, style marking: false, id: 29776, orth form: Fußballspiel, synset id: 21624, sense: 2, source: core, named entity: false, artificial: false, style marking: false, id: 29777, orth form: Fußball, synset id: 21624, sense: 2, source: core, named entity: false, artificial: false, style marking: false]

Every lexical unit has a number of orthographical forms. There are four different orthographical forms but not every 
lexical unit has an entry for all of them:
* main orth. form: 
* orth. variation
* old orth. form
* old orth. variation

We can see that the lexunit for *Fußball* only has one orth form, but that one of its related synsets *Fußballklub* 'football club' has the 
orthographical variation *Fußballkclub*.


In [29]:
LexUnit fussball_unit = germanet.getLexUnitByID(29777);
fussball_unit.getOrthForms();

[Fußball]

In [30]:
LexUnit fussballclub_unit = germanet.getLexUnitByID(32423);
fussballclub_unit.getOrthForms();

[Fußballklub, Fußballclub]

*Fußball* is a compound noun, which are very frequent in the German language. GermaNet stores information about the 
compound, for example that *Fuß* 'foot' is the modifier and *ball* 'ball' is the head.

In [31]:
fussball_unit.getCompoundInfo();


Fuß (Nomen) + Ball

Lexical units are related to other lexical units by different lexical relations. The most common and most general 
lexical relation is synonymy (i.e., the relation which groups lexical units into synsets), but there are other lexical relations in GermaNet as well. For example, for some compounds there has been work
on annotating the relation between the compound and the modifier. In this example the compound *Fußball* has the manner of functioning *Fuß*.


In [32]:
fussball_unit.getRelatedLexUnits(LexRel.has_manner_of_functioning, RelDirection.outgoing);

[id: 35740, orth form: Fuß, synset id: 26149, sense: 3, source: core, named entity: false, artificial: false, style marking: false]

The relations can be unidirectional (e.g. the relation "has manner of functioning" goes from *Fußball*
to *Fuß*, but not the other way around. The relation can also be bidirectional, e.g. *Fußball* and *Fußballspiel* are 
synonyms of each other. If you are interested in finding out which unidirectional relations point towards *Fußball*, these
can be accessed via "incoming_relations":

In [33]:
fussball_unit.getRelatedLexUnits(RelDirection.incoming);

[id: 4226, orth form: fußballerisch, synset id: 2869, sense: 1, source: core, named entity: false, artificial: false, style marking: false, id: 10294, orth form: Fußballschuh, synset id: 7143, sense: 1, source: core, named entity: false, artificial: false, style marking: false, id: 13796, orth form: Fußballstadion, synset id: 9891, sense: 1, source: core, named entity: false, artificial: false, style marking: false, id: 53360, orth form: Fußballamateur, synset id: 37146, sense: 1, source: core, named entity: false, artificial: false, style marking: false, id: 88379, orth form: Fußballschule, synset id: 63191, sense: 1, source: core, named entity: false, artificial: false, style marking: false, id: 29778, orth form: Fußballsport, synset id: 21624, sense: 1, source: core, named entity: false, artificial: false, style marking: false, id: 29776, orth form: Fußballspiel, synset id: 21624, sense: 2, source: core, named entity: false, artificial: false, style marking: false]

Some lexical units have sense definitions, harvested from the German Wictionary. These can be accessed with the wiktionary_paraphrases field.


In [34]:
fussball_unit.getWiktionaryParaphrases();

[LexUnit ID: 29777, Wiktionary ID: 24602, Wiktionary sense definition: Sport, Freizeit, kein Plural: eine beliebte Mannschaftssportart, welche mit 22 Spielern und einem Ball gespielt wird, edited: false]

Some lexical units have also been linked to the English WordNet. The can be accessed with the ili_records field. 


In [35]:
fussball_unit.getIliRecords();

[LexUnit ID: 29777, EWN relation: synonym, PWN word: association football, PWN 2.0 ID: ENG20-00453585-n, PWN 3.0 ID: ENG30-00478262-n, source: initial
English synonyms from PWN 2.0: soccer
English paraphrase from PWN 2.0: a football game in which two teams of 11 players try to kick or head a ball into the opponents' goal]

Lexical units which are verbs provide information on language use by giving at least one example sentence.


In [36]:
LexUnit schiessen = germanet.getLexUnitByID(80272);
schiessen.getExamples();

[Er hatte den Ball ins Tor geschossen., frame: NN.AN.BD]

They are also annotated with subcategorisation patterns / verb complementations (frames). For the given example, the frame specifies that the verb can take accusative complements (AN)

In [37]:
schiessen.getFrames();

[NN.AN.BD]

## How to extract a large number of examples by applying a filter function
If you would like to extract several lexical units or synsets from GermaNet that fulfill certain conditions you can create a filter configuration. For example, filter configurations allow you to search for words of specific
Word Classes (e.g. you might be interested in extracting all abstract nouns) or to extract all words that 
contain a specific subword.

To perform a search you have to create a filter configuration object. You have to pass a search string as an argument. All other options have defaults but you can override these defaults to refine your search.


For example, we can search for *schießen* 'shoot' but ignore upper or lowercasing in different orthforms:

In [38]:
//we can search for "schuss" but we don't want to care about upper or lowercasing and about different orthforms.
//as a default words with all word categories and word classes will be added
FilterConfig filterconfig = new FilterConfig("schießen");
filterconfig.setIgnoreCase(true);
germanet.getSynsets(filterconfig);

[id: 123485, orth forms: [schießen], id: 59153, orth forms: [schießen, knipsen], paraphrases: dilettantisch fotografieren, Bilder machen, id: 21555, orth forms: [Schießsport, Sportschießen, Schießen], id: 56664, orth forms: [schießen], paraphrases: (an einer bestimmten Stelle) mit einem Schuss treffen; mit einem Schuss treffen, id: 60205, orth forms: [schießen, erlegen], paraphrases: Tiere auf der Jagd erlegen; Jägersprache: ein relativ großes Tier bei der Jagd töten, meist durch einen Schuss (Rehe, Hirschen, etc.); (in Bezug auf Wild) erlegen, id: 56650, orth forms: [schießen], paraphrases: einen Schuss/Schüsse abgeben; einen Schuss abgeben, id: 57998, orth forms: [schießen, stürmen, stürzen], paraphrases: mit hoher Geschwindigkeit bewegen, id: 56962, orth forms: [schießen], paraphrases: den Ball mit dem Fuß an eine bestimmte Stelle befördern; einen Ball (mit dem Fuss) fortbewegen]

Let's now limit the results to synsets of a specific semantic class:

In [39]:
filterconfig.setWordClasses(WordClass.Konkurrenz);
germanet.getSynsets(filterconfig);

[id: 56664, orth forms: [schießen], paraphrases: (an einer bestimmten Stelle) mit einem Schuss treffen; mit einem Schuss treffen, id: 56650, orth forms: [schießen], paraphrases: einen Schuss/Schüsse abgeben; einen Schuss abgeben]

If we now filter by word category and use only nouns, our result will be empty because there is no entry for 'schießen' as a noun:

In [40]:
filterconfig.setWordCategories(WordCategory.nomen);
germanet.getSynsets(filterconfig);

[]

Besides using full words as search strings we can use regular expressions. This can be very useful if you are interested 
in words with certain character sequences. The next example shows how to extract all words that end with *kuchen*:


In [41]:
FilterConfig filterconfig = new FilterConfig(".*kuchen");
filterconfig.setRegEx(true);
filterconfig.setWordCategories(WordCategory.nomen);
List<LexUnit> result = germanet.getLexUnits(filterconfig);
System.out.println(String.format("Found  %d words that end with 'kuchen' in GermaNet \n An example of such is: %s \n Another example is : %s"
      , result.size(), result.get(0).toString(), result.get(10).toString()));

Found  54 words that end with 'kuchen' in GermaNet 
 An example of such is: id: 111562, orth form: Buchweizenpfannkuchen, synset id: 82694, sense: 1, source: core, named entity: false, artificial: false, style marking: false 
 Another example is : id: 118233, orth form: Gewürzkuchen, synset id: 87859, sense: 1, source: core, named entity: false, artificial: false, style marking: false


This example extracts all nouns that contain whitespace or a hyphen (useful for example to extract multiword expressions):

In [42]:
// extract all words that contain a white space or a hyphen
FilterConfig filterconfig = new FilterConfig(".+(\\s|-).+");
filterconfig.setRegEx(true);
filterconfig.setWordCategories(WordCategory.nomen);
List<LexUnit> result = germanet.getLexUnits(filterconfig);
System.out.println(String.format("\nFound  %d multiword expressions with whitespace or hypen in GermaNet \n An example of such is: %s \n Another example is : %s"
      , result.size(), result.get(0).toString(), result.get(10).toString()));


Found  5419 multiword expressions with whitespace or hypen in GermaNet 
 An example of such is: id: 15157, orth form: Offenbarung des Johannes, synset id: 10909, sense: 1, source: core, named entity: false, artificial: false, style marking: false 
 Another example is : id: 66741, orth form: Motten-Königskerze, synset id: 46513, sense: 1, source: core, named entity: false, artificial: false, style marking: false


And this example extracts verbs that contain 'ff' or 'ss':   

In [43]:
// extract all verbs that contain exactly two 'ss' or two 'ff'
FilterConfig filterconfig = new FilterConfig(".+(f{2,}|s{2,}).+");
filterconfig.setRegEx(true);
filterconfig.setWordCategories(WordCategory.verben);
List<LexUnit> result = germanet.getLexUnits(filterconfig);
System.out.println(String.format("\nFound  %d verbs with double s or double f in GermaNet \n An example of such is: %s \n Another example is : %s"
      , result.size(), result.get(1).toString(), result.get(10).toString()));


Found  974 verbs with double s or double f in GermaNet 
 An example of such is: id: 159384, orth form: hineinpressen, synset id: 120024, sense: 1, source: core, named entity: false, artificial: false, style marking: false 
 Another example is : id: 194913, orth form: auffalten, synset id: 150207, sense: 2, source: core, named entity: false, artificial: false, style marking: false
